In [1]:
import os
import numpy as np
import import_ipynb
from tqdm import tqdm as tqdm
import edsr

import matplotlib.image as mpimg
from PIL import Image
import cv2
import tensorflow as tf
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_psnr as psnr
import copy

importing Jupyter notebook from edsr.ipynb


/home/chandu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
import tensorflow as tf
import numpy as np


MAX = 1
GAUS_KERNEL = 3
GAUS_SIGMA  = 1.2

def Gaussian_Filter(kernel_size=GAUS_KERNEL, sigma=GAUS_SIGMA):
    
	k = (kernel_size-1)//2 
	filter = []
	sigma_2 = sigma**2
	for i in range(kernel_size):
		filter_row = []
		for j in range(kernel_size):
			Hij = np.exp(-((i+1-(k+1))**2 + (j+1-(k+1))**2)/(2*sigma_2))/(2*np.pi*sigma_2)
			filter_row.append(Hij)
		filter.append(filter_row)
	
	return np.asarray(filter).reshape(kernel_size,kernel_size,1,1)

gaussian_filter = tf.constant(Gaussian_Filter(), tf.float32) 				
h_filter = tf.reshape(tf.constant([[-1,0,1],[-2,0,2],[-1,0,1]], tf.float32), [3,3,1,1])	
v_filter = tf.reshape(tf.constant([[1,2,1],[0,0,0],[-1,-2,-1]], tf.float32), [3,3,1,1])	

np_filter_0 = np.zeros((3,3,1,2))
np_filter_0[1,0,0,0], np_filter_0[1,2,0,1] = 1,1 
filter_0 = tf.constant(np_filter_0, tf.float32)
np_filter_90 = np.zeros((3,3,1,2))
np_filter_90[0,1,0,0], np_filter_90[2,1,0,1] = 1,1 
filter_90 = tf.constant(np_filter_90, tf.float32)
np_filter_45 = np.zeros((3,3,1,2))
np_filter_45[0,2,0,0], np_filter_45[2,0,0,1] = 1,1 
filter_45 = tf.constant(np_filter_45, tf.float32)
np_filter_135 = np.zeros((3,3,1,2))
np_filter_135[0,0,0,0], np_filter_135[2,2,0,1] = 1,1 
filter_135 = tf.constant(np_filter_135, tf.float32)
	
np_filter_sure = np.ones([3,3,1,1]); np_filter_sure[1,1,0,0] = 0
filter_sure = tf.constant(np_filter_sure, tf.float32)
border_paddings = tf.constant([[0,0],[1,1],[1,1],[0,0]])

def Border_Padding(x, pad_width):
	for _ in range(pad_width): x = tf.pad(x, border_paddings, 'SYMMETRIC')
	return x

def FourAngles(d):
	d0   = tf.to_float(tf.greater_equal(d,157.5))+tf.to_float(tf.less(d,22.5))
	d45  = tf.to_float(tf.greater_equal(d,22.5))*tf.to_float(tf.less(d,67.5))
	d90  = tf.to_float(tf.greater_equal(d,67.5))*tf.to_float(tf.less(d,112.5))
	d135 = tf.to_float(tf.greater_equal(d,112.5))*tf.to_float(tf.less(d,157.5))
	return (d0,d45,d90,d135)


def TF_Canny(img_tensor, minRate=0.10, maxRate=0.40,preserve_size=True, remove_high_val=False, return_raw_edges=False):
	img_tensor = (img_tensor)*MAX
	if preserve_size: img_tensor = Border_Padding(img_tensor, (GAUS_KERNEL-1)//2)

	x_gaussian = tf.nn.convolution(img_tensor, gaussian_filter, padding='VALID')
	if remove_high_val: x_gaussian = tf.clip_by_value(x_gaussian, 0, MAX/2)
	

	if preserve_size: x_gaussian = Border_Padding(x_gaussian, 1)
	Gx = tf.nn.convolution(x_gaussian, h_filter, padding='VALID')
	Gy = tf.nn.convolution(x_gaussian, v_filter, padding='VALID')
	G 		= tf.sqrt(tf.square(Gx) + tf.square(Gy))
	BIG_PHI = tf.atan2(Gy,Gx)
	BIG_PHI	= (BIG_PHI*180/np.pi)%180 		
	D_0,D_45,D_90,D_135 = FourAngles(BIG_PHI)	
	
	
	targetPixels_0 = tf.nn.convolution(G, filter_0, padding='SAME')
	isGreater_0 = tf.to_float(tf.greater(G*D_0, targetPixels_0))
	isMax_0 = isGreater_0[:,:,:,0:1]*isGreater_0[:,:,:,1:2]

	targetPixels_90 = tf.nn.convolution(G, filter_90, padding='SAME')
	isGreater_90 = tf.to_float(tf.greater(G*D_90, targetPixels_90))
	isMax_90 = isGreater_90[:,:,:,0:1]*isGreater_90[:,:,:,1:2]
	
	targetPixels_45 = tf.nn.convolution(G, filter_45, padding='SAME')
	isGreater_45 = tf.to_float(tf.greater(G*D_45, targetPixels_45))
	isMax_45 = isGreater_45[:,:,:,0:1]*isGreater_45[:,:,:,1:2]
	
	targetPixels_135 = tf.nn.convolution(G, filter_135, padding='SAME')
	isGreater_135 = tf.to_float(tf.greater(G*D_135, targetPixels_135))
	isMax_135 = isGreater_135[:,:,:,0:1]*isGreater_135[:,:,:,1:2]

	edges_raw = G*(isMax_0 + isMax_90 + isMax_45 + isMax_135)
	edges_raw = tf.clip_by_value(edges_raw, 0, MAX)
  
	if return_raw_edges: return tf.squeeze(edges_raw)
	
	edges_sure = tf.to_float(tf.greater_equal(edges_raw, maxRate))
	edges_weak = tf.to_float(tf.less(edges_raw, maxRate))*tf.to_float(tf.greater_equal(edges_raw, minRate))
	
	edges_connected = tf.nn.convolution(edges_sure, filter_sure, padding='SAME')*edges_weak
	for _ in range(10): edges_connected = tf.nn.convolution(edges_connected, filter_sure, padding='SAME')*edges_weak
	
	edges_final = edges_sure + tf.clip_by_value(edges_connected,0,MAX)
	return tf.squeeze(edges_final)

def edge_loss(y_true, y_pred):
    y_true = tf.cast(y_true,dtype = tf.float32)
    y_pred = tf.cast(y_pred,dtype = tf.float32)
    E = TF_Canny(y_true, return_raw_edges=False)
    E = 1 - E
    E = tf.cast(E,dtype = tf.float32)
    K.print_tensor(E)
    A = y_true - y_pred
    A = tf.cast(A,dtype = tf.float32)
    E = tf.reshape(E,(10,256,256,1))
    K.print_tensor(E)
    LP = K.abs(A)
    LP = tf.cast(LP,dtype = tf.float32)
    SQ = K.mean(K.square(y_pred - y_true), axis=-1)
    
    LE = tf.multiply(E,LP)
    LE = tf.cast(LE,dtype = tf.float32)
    
    SQ1 = K.mean(K.square(LE), axis=-1)
    
    los = 0.7 * SQ + 0.3 * SQ1
    return los

In [3]:
from keras.layers import Conv2D

def compact(x_in):
    conv1 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(x_in)
    conv2 = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(conv1)
    last = Conv2D(1, strides = 2, kernel_size=(3,3), activation='sigmoid', padding = 'same')(conv2)
    return last

In [4]:
def compress_this(x_input, quality_factor):
    
    num = x_input.shape[0]
    ans = []
    for i in range(num):
        x_single = x_input[i,:,:,0]
        img = Image.fromarray(np.uint8(x_single*255))
        img.save(os.getcwd()+'/com4.JPEG',"JPEG", quality = quality_factor)
        out_single = mpimg.imread(os.getcwd()+'/com4.JPEG')
        ans.append(out_single)
    ans = np.array(ans)
    ans = np.reshape(ans, (ans.shape[0], ans.shape[1], ans.shape[2], 1))
    return ans

In [5]:
def ycbcr2rgb(img):
    img = cv2.cvtColor(img, cv2.COLOR_YCR_CB2RGB)
    return img

In [6]:
def load_images(dset_location):
    loc = dset_location
    images_location = []
    X = []

    print("Extracting image locations..")
    for i in tqdm(os.listdir(loc)):
        images_location.append(loc+'/'+i)

    print("Extracting images..")
    for im_loc in tqdm(images_location):
        img = cv2.imread(im_loc)
        x, y = img.shape[0], img.shape[1]
        if x%2!=0:
            x-=1
        if y%2!=0:
            y-=1
        img = cv2.resize(img, (x, y))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
        X.append(img)
    X = np.array(X)
    return X

In [10]:
from keras.layers import Input
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam


inp = Input(shape=(None, None, 1))
model_edsr = edsr.edsr(x_in=inp, scale=2)
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=0.00000001, decay=0.0, amsgrad=False)
model_edsr.compile(optimizer=adam, loss='mae')
model_edsr.summary()


inp = Input(shape=(None, None, 1))
model_comcnn = Model(inp, model_edsr(compact(inp)))
model_comcnn.layers[4].trainable = False
model_comcnn.compile(optimizer='adam', loss=edge_loss)
model_comcnn.summary()

model_edsr.load_weights('edsr.weights.best.hdf5')
model_comcnn.load_weights('comcnn.weights.best.hdf5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, None, None, 6 640         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, None, None, 6 36928       conv2d_23[0][0]                  
__________________________________________________________________________________________________
activation_9 (Activation)       (None, None, None, 6 0           conv2d_24[0][0]                  
__________________________________________________________________________________________________
conv2d_25 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, None, None, 64)    640       
_________________________________________________________________
conv2d_43 (Conv2D)           (None, None, None, 32)    18464     
_________________________________________________________________
conv2d_44 (Conv2D)           (None, None, None, 1)     289       
_________________________________________________________________
edsr (Model)                 (None, None, None, 1)     776705    
Total params: 796,098
Trainable params: 19,393
Non-trainable params: 776,705
_________________________________________________________________


In [2]:
def run():
    
    in_loc = 'test_images'
    out_loc = 'output_images'
    val_psnr = []
    val_ssim = []
    
    for im_loc in (os.listdir(in_loc)):
        adr = in_loc + '/' + im_loc

        X = []
        img = cv2.imread(adr)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
        m, n = img.shape[0], img.shape[1]

        flag = 0
        if m%2!=0:
            flag = 1
            m = m - 1

        if n%2!=0:
            flag = 1
            n = n - 1

        img = cv2.resize(img, (m, n))

        X.append(img)
        X = np.array(X)
        X = X.astype('float32')
        X = X / 255

        mpimg.imsave(out_loc+'/'+im_loc+'-in.png', ycbcr2rgb(X[0]), cmap = 'gray')
        upto_comcnn = Model(model_comcnn.input, model_comcnn.layers[3].output)
        Ycom = upto_comcnn.predict(X[:,:,:,:1])

        Xcompress = compress_this(Ycom, 10)
        Xcompress = Xcompress.astype('float32')
        Xcompress = Xcompress / 255

        Y = model_edsr.predict(Xcompress)
        Y[Y>1] = 1
        Z = copy.deepcopy(X)
        Z[:,:,:,0] = Y[:,:,:,0]
        mpimg.imsave(out_loc+'/'+im_loc+'-out.png', ycbcr2rgb(Z[0]), cmap = 'gray')
        val_psnr.append(psnr(Y[0,:,:,0], X[0,:,:,0]))
        val_ssim.append(ssim(Y[0,:,:,0], X[0,:,:,0]))
        print(im_loc + " : ", psnr(Y[0,:,:,0], X[0,:,:,0]))
        print(im_loc + " : ", ssim(Y[0,:,:,0], X[0,:,:,0]))

    print("Average PSNR : ", np.mean(val_psnr))
    print("Average SSIM : ", np.mean(val_ssim))